In [1]:
import numpy as np
import torch
from utils.libmise.mise import  MISE
from utils.libmcubes.mcubes import marching_cubes
import trimesh
import os

In [72]:
padding = 0.1
threshold_g = 0.2

In [73]:
def make_3d_grid(bb_min, bb_max, shape):
    ''' Makes a 3D grid.

    Args:
        bb_min (tuple): bounding box minimum
        bb_max (tuple): bounding box maximum
        shape (tuple): output shape
    '''
    size = shape[0] * shape[1] * shape[2]

    pxs = torch.linspace(bb_min[0], bb_max[0], shape[0])
    pys = torch.linspace(bb_min[1], bb_max[1], shape[1])
    pzs = torch.linspace(bb_min[2], bb_max[2], shape[2])

    pxs = pxs.view(-1, 1, 1).expand(*shape).contiguous().view(size)
    pys = pys.view(1, -1, 1).expand(*shape).contiguous().view(size)
    pzs = pzs.view(1, 1, -1).expand(*shape).contiguous().view(size)
    p = torch.stack([pxs, pys, pzs], dim=1)

    return p

In [93]:
def extract_mesh(occ_hat):
    n_x, n_y, n_z = occ_hat.shape
    box_size = 1 + padding
    threshold = np.log( threshold_g) - np.log(1. - threshold_g)
    
    occ_hat_padded = np.pad(occ_hat, 1, 'constant', constant_values=-1e6)
    
    vertices, triangles = marching_cubes(occ_hat_padded, threshold)
    print(triangles)
    
    vertices -= 0.5
    # Undo padding
    vertices -= 1
    # Normalize to bounding box
    vertices /= np.array([n_x-1, n_y-1, n_z-1])
    vertices = box_size * (vertices - 0.5)
    
    normals = None

    # Create mesh
    mesh = trimesh.Trimesh(vertices, triangles, vertex_normals=normals,process=False)

    return mesh

In [94]:
def get_mesh(occ, points,threshold = 0.5,padding=0.1,resolution0=16,upsampling_steps=3):
    threshold = np.log(threshold_g) - np.log(1. - threshold_g)
    
    nx = 32
    pointsf = 2 * make_3d_grid((-0.5,)*3, (0.5,)*3, (nx,)*3    )
    
    value_grid = occ.reshape(nx, nx, nx)
     
    mesh = extract_mesh(value_grid)

    return mesh

In [95]:
occ_file = '../sample_data/points/occupancies.npy'
points_file = '../sample_data/points/points.npy'

points = np.load(points_file)

occ = np.load(occ_file)
occ = np.unpackbits(occ)

idx = np.random.choice(np.arange(100000), 32768, replace=False)
occ_sample = occ[idx]
points_sample = points[idx]



mesh = get_mesh(occ_sample,points_sample)


mesh_out_file = os.path.join('./', '%s.off' % 'onet')
mesh.export(mesh_out_file)

[]


'OFF\n0 0 0\n\n'